In [ ]:
import datetime as dt

import pandas as pd
import tinvest
from tinvest import InstrumentType, OperationTypeWithCommission, Currency, OperationStatus

In [ ]:
TOKEN = "insert your token here"
client = tinvest.SyncClient(TOKEN)

In [ ]:
accounts = client.get_accounts()
broker_account_id = [
    brocker_account['broker_account_id']
    for brocker_account in accounts.payload.dict()['accounts'] 
    if brocker_account['broker_account_type'].value == 'TinkoffIis'
][0]

In [ ]:
ops_from = dt.datetime.now() - dt.timedelta(days=2 * 365)
ops_to = dt.datetime.now()
ops = client.get_operations(from_=ops_from, to=ops_to, broker_account_id=broker_account_id)

In [ ]:
ops_df = pd.DataFrame(ops.payload.dict()['operations']).set_index('id')
# преобразование даты
ops_df['date'] = pd.to_datetime(ops_df['date'])
ops_df['date'] = ops_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
# оставляем только успешные п=операции 
ops_df = ops_df[ops_df['status'] == OperationStatus.done]
# учитываем покупки только в валюте RUB 
ops_df = ops_df[ops_df['currency'] == Currency.rub]
# обрабатываем сумму комиссии 
ops_df['commission'] = ops_df['commission'].apply(lambda x: x['value'] if x is not None else None)
# учитываем только операции покупок, родаж и т.д. (минуя пополнения)
ops_df = ops_df[ops_df['instrument_type'].notna()]
# преобразвование enum -> str
ops_df['instrument_type'] = ops_df['instrument_type'].apply(lambda x: x.name)
ops_df['operation_type'] = ops_df['operation_type'].apply(lambda x: x.name)
# откидываем доп. операции списания комиссии брокера
ops_df = ops_df[ops_df['operation_type'] != 'broker_commission']
# удаляем лишние колонки
ops_df = ops_df.drop(columns=['status', 'trades', 'quantity', 'currency', 'is_margin_call'])
# форматирвоание данных: переименовываем колонки в понятные формат и сортируем по дате
ops_df = ops_df.rename(columns={'date': 'dt', 'price': 'unit_price', 'payment': 'total_price', 'quantity_executed': 'count', 'commission': 'commission'})
ops_df = ops_df.sort_values(by='dt')
# сопоставляем кадома FIGI его ticker
figi_to_ticker = {figi: client.get_market_search_by_figi(figi).payload.ticker for figi in ops_df['figi'].unique()}
ops_df['ticker'] = ops_df['figi'].map(figi_to_ticker)
# форматирование днных: колонки в нужном формате
ops_df = ops_df[['dt', 'ticker', 'figi', 'instrument_type', 'operation_type', 'total_price', 'unit_price', 'count', 'commission']]
ops_df.head()

In [ ]:
ops_df.to_csv('data/tinkoff/operations.csv', index=True, header=True, encoding='utf-8')